In [1]:
import sys
import os
sys.path.append(os.path.abspath('..'))  

In [2]:
import pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as sci_plt
import csv  
from pprint import pprint
from sklearn.preprocessing import StandardScaler
from operations.fetch_data import PriceHistory
from keys.all_keys import alpha_vantage_api_key

In [3]:
df = pd.read_csv('C:\projects\Son_of_anton\data\stocks.csv')
df.head()


,Unnamed: 0,date,open,high,low,close,volume,symbol,daily_return,price_range,avg_price
0,0,1999-11-01,80.00,80.69,77.37,77.62,2487300,AAPL,NaN,3.32,78.560000
1,1,1999-11-02,78.00,81.69,77.31,80.25,3564600,AAPL,0.033883,4.38,79.750000
2,2,1999-11-03,81.62,83.25,81.00,81.50,2932700,AAPL,0.015576,2.25,81.916667
3,3,1999-11-04,82.06,85.37,80.62,83.62,3384700,AAPL,0.026012,4.75,83.203333
4,4,1999-11-05,84.62,88.37,84.00,88.31,3721500,AAPL,0.056087,4.37,86.893333


In [4]:
# grab the cols we need
price_df = df[['date', 'symbol', 'close']]


#pivot the df to make the symbols the headers
price_df = price_df.pivot(
    index= 'date',
    columns='symbol',
    values='close'
)

price_df


symbol,AAPL,AMZN,BAC,GOOGL,JPM,MSFT,QQQ,SPY,VTI,WFC
date,,,,,,,,,,
1999-11-01,77.62,69.13,64.87,NaN,83.56,92.37,130.80,135.5625,NaN,46.88
1999-11-02,80.25,66.44,64.25,NaN,83.69,92.56,130.90,134.5937,NaN,47.38
1999-11-03,81.50,65.81,63.00,NaN,82.44,92.00,133.50,135.5000,NaN,46.13
1999-11-04,83.62,63.06,63.50,NaN,84.12,91.75,135.00,136.5312,NaN,47.00
1999-11-05,88.31,64.94,65.06,NaN,86.25,91.56,136.40,137.8750,NaN,46.81
...,...,...,...,...,...,...,...,...,...,...
2025-06-05,200.63,207.91,44.38,168.21,261.95,467.68,524.79,593.0500,291.72,74.90
2025-06-06,203.92,213.57,44.97,173.68,265.73,470.38,529.92,599.1400,295.12,76.33
2025-06-09,201.45,216.98,44.87,176.09,266.74,472.75,530.70,599.6800,295.36,76.46


In [5]:
# calculate the log return
log_return = np.log(1+price_df.pct_change())
#print('log_return:', log_return)

#calculating the number of symbols
number_of_symbols = len(df['symbol'].unique())

# weights (randomly assigned)
random_weights = np.array(np.random.random(number_of_symbols))
print('random_weights:', random_weights)

#rebalance weights (must be equals to 1)
rebalanced_weights = random_weights/np.sum(random_weights)
print('rebalanced_weights:', rebalanced_weights)

#calc expected Annualized returns (multipiled with 252 for annualization)
exp_returns =  np.sum((log_return.mean()*rebalanced_weights)*252)
print('expected returns:', exp_returns)

#calc expected volality annualized
exp_volt = np.sqrt(np.dot(rebalanced_weights.T,
                          np.dot(log_return.cov()*252,
                                 rebalanced_weights)))
print('exp_volt:', exp_volt)

#cal the sharp ratio
risk_free_rate = 0.03 # us tresury yield 2025
print('r_f:', risk_free_rate)

sharp_ratio = (exp_returns-risk_free_rate)/exp_volt 
print('sharp_ratio:', sharp_ratio)




random_weights: [0.42069843 0.99202625 0.60775547 0.9394532  0.59976523 0.91392646
 0.11772768 0.15224717 0.01004839 0.73542306]
rebalanced_weights: [0.07664292 0.18072752 0.110721   0.17114975 0.10926534 0.16649929
 0.02144765 0.02773642 0.00183062 0.13397951]
expected returns: 0.03480888946131262
exp_volt: 0.31446743604983624
r_f: 0.03
sharp_ratio: 0.015292169903883211


In [6]:
def get_metrics(weights:list, risk_free_rate:float) -> np.array:
    # convert weights to np.array()
    weights = np.array(weights)

    # calc returns
    returns = np.sum(log_return.mean()*weights)*252

    #calc volatility
    vol = np.sqrt(np.dot(weights.T, np.dot(log_return.cov()*252, weights)))

    # calc the sharpe ratio
    sharp_ratio = (returns - risk_free_rate)/vol

    return np.array([returns, vol, sharp_ratio])



In [7]:
def grab_neg_sharpe_ratio(weights: list) -> np.array:
    '''
    we don't have a way to maximize sharpe ratio in scipy; so we are minimizing the neg sharp ratio
    '''
    return get_metrics(weights=weights, risk_free_rate=risk_free_rate)[2]-1



In [8]:
def grab_vol(weights:list)->np.array:
    return get_metrics(weights=weights, risk_free_rate=risk_free_rate)[1]


In [9]:
def check_sum(weights:list)->float:
    return np.sum(weights) - 1

In [10]:
 # define bounds of our optimization process
bounds = tuple((0,1) for symbol in range(number_of_symbols))

constraints = ({'type': 'eq', 'fun': check_sum})

init_weights = number_of_symbols * [1/number_of_symbols]

optimized_sharpe = sci_plt.minimize(
    grab_neg_sharpe_ratio,
    init_weights,
    method='SLSQP',
    bounds=bounds,
    constraints=constraints
)

print('')
print('=' * 250)
print('Optimal sharpe ratio:')  
print('-' * 250)
print(optimized_sharpe)
print('-' * 250)


Optimal sharpe ratio:
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -1.0976944860092521
       x: [ 0.000e+00  0.000e+00  1.000e+00  1.152e-16  5.478e-17
            1.383e-17  2.273e-17  1.659e-17  6.559e-19  0.000e+00]
     nit: 7
     jac: [ 1.062e-01  1.229e-01  6.580e-02  8.525e-02  1.597e-01
            1.628e-01  1.455e-01  1.539e-01  1.135e-01  1.021e-01]
    nfev: 77
    njev: 7
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


In [11]:
optimized_metrix =  get_metrics(weights=optimized_sharpe.x, risk_free_rate=0.03)

print('')
print('=' * 250)
print('Optimalized Matrix:')  
print('-' * 250)
print(optimized_metrix)
print('-' * 250)

print('')
print('=' * 250)
print('Optimalized weights:')  
print('-' * 250)
print(optimized_sharpe.x)
print('-' * 250)


Optimalized Matrix:
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
[-0.01454412  0.45595329 -0.09769449]
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Optimalized weights:
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
[0.00000000e+00 0.00000000e+00 1.00000000e+00 1.15173801e-16
 5.47780784e-17 1.38306334e-17 2.27304521e-17 1.65939905e-17
 6.55937434e-19 0.00000000e+00]
------------

In [12]:
 # define bounds of our optimization process
bounds = tuple((0,1) for symbol in range(number_of_symbols))

constraints = ({'type': 'eq', 'fun': check_sum})

init_weights = number_of_symbols * [1/number_of_symbols]

optimized_vol = sci_plt.minimize(
    grab_vol,
    init_weights,
    method='SLSQP',
    bounds=bounds,
    constraints=constraints
)

print('')
print('=' * 250)
print('Optimal volatility:')  
print('-' * 250)
print(optimized_vol)
print('-' * 250)


Optimal volatility:
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.1952999791789375
       x: [ 0.000e+00  2.991e-17  4.944e-17  0.000e+00  5.368e-17
            9.353e-17  1.378e-18  9.520e-01  4.801e-02  2.051e-17]
     nit: 10
     jac: [ 2.228e-01  2.397e-01  2.847e-01  1.965e-01  2.671e-01
            2.083e-01  2.307e-01  1.953e-01  1.953e-01  2.397e-01]
    nfev: 110
    njev: 10
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


In [13]:
optimized_vol_metrix =  get_metrics(weights=optimized_vol.x, risk_free_rate=0.03)

print('')
print('=' * 250)
print('Optimalized vol Matrix:')  
print('-' * 250)
print(optimized_vol_metrix)
print('-' * 250)

print('')
print('=' * 250)
print('Optimalized weights:')  
print('-' * 250)
print(optimized_vol.x)
print('-' * 250)


Optimalized vol Matrix:
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
[0.05738487 0.19529998 0.14021951]
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Optimalized weights:
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
[0.00000000e+00 2.99083697e-17 4.94439795e-17 0.00000000e+00
 5.36781664e-17 9.35278174e-17 1.37820255e-18 9.51992859e-01
 4.80071408e-02 2.05142883e-17]
-----------

In [14]:
rounded_weights = [round(w, 4) for w in optimized_vol.x]

print('')
print('=' * 250)
print('Optimized vol Matrix:')  
print('-' * 250)
print(optimized_vol_metrix)
print('-' * 250)

print('')
print('=' * 250)
print('Optimized weights:')  
print('-' * 250)
print(rounded_weights)
print('-' * 250)



Optimized vol Matrix:
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
[0.05738487 0.19529998 0.14021951]
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Optimized weights:
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
[np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.952), np.float64(0.048), np.float64(